# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

### Executed on Google Colab servers [here](https://colab.research.google.com/drive/1Xn-qh1mYlMifeIh9Oi_DdColGJEUnOMZ)

## 0. Prepare data

In [1]:
!wget http://www.gutenberg.org/files/2600/2600-0.txt
!mv 2600-0.txt peace.txt

--2019-10-10 18:53:08--  http://www.gutenberg.org/files/2600/2600-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3359549 (3.2M) [text/plain]
Saving to: ‘2600-0.txt’

2600-0.txt          100%[===================>]   3.20M  1.46MB/s    in 2.2s    

2019-10-10 18:53:10 (1.46 MB/s) - ‘2600-0.txt’ saved [3359549/3359549]



In [2]:
text = open('peace.txt', 'r').read()[2:]
len(text)

3227579

## 1. Make text lowercase and remove all punctuation except spaces and dots.

In [0]:
import re

In [0]:
def preprocess_text(text):
    text = text.lower().replace("\n", " ")
    text = re.sub(r"[^.' \-\w]", "", text)
    text = re.sub(r"\.(?!\s)", "", text)
    text = re.sub(r" +", " ", text)
    return text

In [5]:
%%time

text = preprocess_text(text)

CPU times: user 235 ms, sys: 31.7 ms, total: 267 ms
Wall time: 269 ms


In [6]:
len(text)

3128031

In [0]:
text = text.split('.')
text = [x.strip() for x in text]

In [8]:
len(text)

26766

## 2. Tokenize text by BPE with vocab_size = 100

In [0]:
from collections import Counter, OrderedDict
from sklearn.base import TransformerMixin

In [0]:
class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        self.stok = OrderedDict()
        self.ktos = OrderedDict()
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        uniquechars = set(list("".join(text)))
        for c in uniquechars:
          self.stok[c] = c
        
        while len(self.stok) < self.vocab_size:
            new_count = [s[i]+s[i+1] for s in text for i in range(len(s)-1)]
            new_count = Counter(new_count)
            new_token = new_count.most_common(1)[0][0]
            new_ch = chr(max([ord(c) for c in list(self.stok.values())]) + 1)
            
            self.stok[new_token] = new_ch
            
            text = [s.replace(new_token, new_ch) for s in text]
        
        for k in self.stok:
          self.ktos[self.stok[k]] = k
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        gluer = ord("\n")
        used = set([ord(ch) for ch in self.stok.values()])
        for s in text:
          used.update(set([ord(ch) for ch in s]))
        if gluer in used:
          gluer = max(list(used)) + 1
        gluer = chr(gluer)
        text = gluer.join(text)
        for subst in self.stok:
          if subst != self.stok[subst]:
            text = text.replace(subst, self.stok[subst])
        text = text.split(gluer)
        text = [[ord(ch) for ch in s] for s in text]
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        if isinstance(tok, int):
          k = chr(tok)
          if k == self.ktos[k]:
            return k
          else:
            return self.decode_token(tuple(ord(ch) for ch in self.ktos[k]))
        elif isinstance(tok, tuple):
          return "".join([self.decode_token(k) for k in tok])
        else:
          raise TypeError(" ".join(["decode_token() accepts only str and tuple",
                          "instances, not", str(type(tok))]))
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))

In [11]:
%%time

vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

CPU times: user 27.3 s, sys: 1.38 s, total: 28.7 s
Wall time: 28.8 s


In [12]:
len(tokenized_text)

26766

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

## 3. Train 3-gram language model with laplace smoothing δ=1

In [0]:
import math
from itertools import product

In [0]:
class LM:
    def __init__(self, vocab_size, start_id, end_id, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.infer_struct = {}
        self.proba = self.infer_struct
        self.start_id = start_id
        self.start_token = chr(start_id)
        self.end_id = end_id
        self.end_token = chr(end_id)


    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = {ord(ch): self.get_proba(a, b, ch, tau) for ch in self.vocab}
        return result

    def get_raw_proba(self, a, b, c, tau=1):
        smoothed_proba = self.get_proba(a, b, c, tau)
        return pow(math.e, smoothed_proba)

    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        if not isinstance(a, str):
          if isinstance(a, int):
            a = chr(a)
          else:
            return TypeError("Only chars and integers accepted as indices")
        if len(a) != 1:
          return ValueError("String indices must be chars (have length 1)")
        if a not in self.vocab:
          return ValueError("Unknown token: "+a+" (ord value: "+str(ord(a))+")")
        if not isinstance(b, str):
          if isinstance(b, int):
            b = chr(b)
          else:
            return TypeError("Only chars and integers accepted as indices")
        if len(b) != 1:
          return ValueError("String indices must be chars (have length 1)")
        if b not in self.vocab:
          return ValueError("Unknown token: "+b+" (ord value: "+str(ord(b))+")")
        if not isinstance(c, str):
          if isinstance(c, int):
            c = chr(c)
          else:
            return TypeError("Only chars and integers accepted as indices")
        if len(c) != 1:
          return ValueError("String indices must be chars (have length 1)")
        if c not in self.vocab:
          return ValueError("Unknown token: "+c+" (ord value: "+str(ord(c))+")")
        
        b_prob = pow(self.infer_struct[a][b]["total"] + self.delta*len(self.vocab), 1/tau)
        t_prob = pow(self.infer_struct[a][b][c] + self.delta, 1/tau)
        result = math.log(t_prob/b_prob)
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        bigram_count = Counter()
        trigram_count = Counter()
        self.vocab = set()
        for el in text:
          s = self.start_token + "".join([chr(k) for k in el]) + self.end_token
          self.vocab.update(set(s))
          bigram_count += Counter([s[i:i+2] for i in range(len(s)-1)])
          trigram_count += Counter([s[i:i+3] for i in range(len(s)-2)])
        bigram_hits = {}
        bigrams = ["".join(list(res)) for res in product(self.vocab, self.vocab)]
        for bigram in bigrams:
          bigram_hits[bigram] = bigram_count[bigram]
        trigram_hits = {}
        trigrams = ["".join(list(res)) for res in product(self.vocab, self.vocab, self.vocab)]
        for trigram in trigrams:
          trigram_hits[trigram] = trigram_count[trigram]
        self.infer_struct = {}
        for a in self.vocab:
          self.infer_struct[a] = {}
          for b in self.vocab:
            self.infer_struct[a][b] = {}
            self.infer_struct[a][b]["total"] = bigram_hits[a+b]
            for c in self.vocab:
              self.infer_struct[a][b][c] = trigram_hits[a+b+c]

        return self

In [0]:
start_token = max(ord(ch) for ch in bpe.ktos.keys()) + 1
end_token = start_token + 1

In [17]:
%%time

lm = LM(vocab_size, start_token, end_token, 1).fit(tokenized_text)

CPU times: user 1min 3s, sys: 76.6 ms, total: 1min 3s
Wall time: 1min 3s


## 4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    
    def get_rays(keys_dict, lm, k, tau):
      res_dict = {}
      for ks in keys_dict:
        res = lm.infer(ks[-2], ks[-1], tau)
        res_dict.update({ks+chr(ky): res[ky] for ky in res})
      return {ks: res_dict[ks] + keys_dict[ks[:-1]]
              for ks in sorted(res_dict, key=res_dict.get, reverse=True)[:k]}

    if len(input_seq) < 2:
      raise ValueError("input_seq should be at least 2 elements long, got " +
                       str(input_seq) + " instead")

    start_id = lm.start_id
    end_id = lm.end_id

    if input_seq[0] != start_id:
      input_seq = [start_id] + input_seq
      max_len += 1

    input_seq = "".join([chr(ch) for ch in input_seq])

    if len(input_seq) == 2:
      query = lm.infer(input_seq[0], input_seq[1], tau)
      proba = query[sorted(query, key=query.get, reverse=True)[0]]
    else:
      proba = lm.get_proba(input_seq[0], input_seq[1], input_seq[2], tau)
      for i in range(3, len(input_seq)):
        proba += lm.get_proba(input_seq[i-2], input_seq[i-1], input_seq[i], tau)

    best_matches = []
    current_beam = {input_seq: proba}
    
    for i in range(len(input_seq), max_len):
        new_beam = get_rays(current_beam, lm, k, tau)
        for m in new_beam:
          if m.endswith(chr(end_id)):
            best_matches.append([m, new_beam[m]])
        current_beam = {m: new_beam[m] for m in new_beam
                        if not m.endswith(chr(end_id))}
        if len(best_matches) == k:
          break
    
    final_matches = sorted(current_beam, key=current_beam.get, reverse=True)
    i = 0

    while len(best_matches) < k:
      best_matches.append([final_matches[i], current_beam[final_matches[i]]])
      i += 1

    best_matches = [[[ord(ch) for ch in
                      m[0].replace(chr(start_id), "").replace(chr(end_id), "")],
                     m[1]] for m in best_matches]

    return best_matches

In [19]:
input1 = 'any'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)

for sequence, proba in result:
  print(bpe.decode(sequence), proba, sep=":\t")

any:	-74.35749344668162
anya:	-109.41881619508852
anyone:	-87.49836010663222
anywho had been:	-151.25910593581406
anywho had beg:	-155.02020691961656
anywho had bef:	-156.73747682324216
anyoney who had :	-159.71480566880385
anybold been :	-163.39228343621178
anyoney who was :	-166.1324078055385
anybothere who :	-173.5104235712026


In [20]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)

for sequence, proba in result:
  print(bpe.decode(sequence), proba, sep=":\t")

horse prince:	-151.2079940555816
horse said no:	-143.8161743689936
horse princes:	-144.3044552290383
horse prince an:	-144.6350576832909
horse who were :	-150.5379091721297
horse princes :	-150.6387346876215
horse who had b:	-152.68927425123724
horse they were :	-155.69354562382085
horse they who :	-169.4443898136467
horse they when:	-172.87277684578956


In [21]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)

for sequence, proba in result:
  print(bpe.decode(sequence), proba, sep=":\t")

her:	-40.27213839409248
here:	-67.1249462801872
hers:	-59.37099832287781
heroom:	-91.99568974416749
herience:	-141.09601007054923
herence andrew :	-135.36956219788482
herkónya did :	-147.54892237511427
herience andrew:	-149.8819160036072
herkónskill :	-150.78125079375124
herence andress:	-151.30963155414918


In [22]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)

for sequence, proba in result:
  print(bpe.decode(sequence), proba, sep=":\t")

what:	-7.588823296598268
whats:	-9.683769024814069
whatself:	-12.819002208709813
whatimpers:	-16.381394482242
whatraid not :	-13.397609923140392
whatimperself :	-16.832177198515293
whatched the could :	-17.238741276340953
whatself-cam:	-17.613900401193156
whatself-cau:	-17.951896641157344
whatched the cound :	-18.049154477895215


In [23]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)

for sequence, proba in result:
  print(bpe.decode(sequence), proba, sep=":\t")

gun proom:	-189.18812874564293
gun prince:	-179.02182208837888
gun princes:	-172.1182832618356
gun prince an:	-172.44888571608823
gun prostóv:	-177.42428183273356
gun princes :	-178.4525627204188
gun who had b:	-181.27028172654283
gun they were :	-183.915812926423
gun proom the :	-194.4350320724103
gun themperor:	-195.22498170630126


## 5. Calculate perplexity of the language model for the first sentence.

In [0]:
def perplexity(snt, lm, raw=False):
    """
    snt: sequence of token ids
    lm: language model
    """
    if len(snt) < 2:
      raise ValueError("snt should be at least 2 elements long, got " +
                       str(snt) + " instead")

    if len(snt) == 2:
      query = lm.infer(snt[0], snt[1])
      proba = query[sorted(query, key=query.get, reverse=True)[0]]
      if raw:
        proba = pow(proba, math.e)
    else:
      if raw:
        proba = lm.get_raw_proba(snt[0], snt[1], snt[2])
      else:
        proba = lm.get_proba(snt[0], snt[1], snt[2])
      for i in range(3, len(snt)):
        if raw:
          proba += lm.get_raw_proba(snt[i-2], snt[i-1], snt[i])
        else:
          proba += lm.get_proba(snt[i-2], snt[i-1], snt[i])
    return proba

In [25]:
perplexity(tokenized_text[0], lm)

-268.46633741055825